In [ ]:
import numpy as np
import pandas as pd

class TSP(object):
    citys = np.array([])
    citys_name = np.array([])
    
    pop_size = 50
    c_rate = 0.7
    m_rate = 0.05
    
    pop = np.array([])
    fitness = np.array([])
    
    city_size = -1
    ga_num = 200
    
    best_dist = -1
    best_gem = []
    dw = Draw()
    
    def __init__(self, c_rate, m_rate, pop_size, ga_num):
        self.fitness = np.zeros(self.pop_size)
        self.c_rate = c_rate
        self.m_rate = m_rate
        self.pop_size = pop_size
        self.ga_num = ga_num
        
    def init(self):
        tsp = self
        tsp.load_citys()
        tsp.pop = tsp.create_pop(tsp.pop_size)
        tsp.fitness = tsp.get_fitness(tsp.pop)
        tsp.dw.bound_x = [np.min(tsp.citys[:, 0]), np.max(tsp.citys[:, 0])]
        tsp.dw.bound_y = [np.min(tsp.citys[:, 1]), np.max(tsp.citys[:, 1])]
        tsp.dw.set_xybound(tsp.dw.bound_x, tsp.dw.bound_y)
    
    def load_citys(self):
        data = pd.read_csv(file = 'china.csv', delimiter = ';', header = None).values
        self.citys = data[:, 1:]
        self.citys_name = data[:, 0]
        self.city_size = data.shape[0]
    
    def create_pop(self, size):
        pop = []
        for i in range(size):
            gene = np.arange(self.citys.shape[0])
            np.random.shuffle(gene)
            pop.append(gene)
        return np.array(pop)
    
    def evolution(self):
        tsp = self
        for i in range(self.ga_num):
            best_f_index = np.argmax(tsp.fitness)
            worst_f_index = np.argmin(tsp.fitness)
            local_best_gen = tsp.pop[best_f_index]
            local_best_dist = tsp.gen_distance(local_best_gen)
            
            if i == 0:
                tsp.best_gen = local_best_gen
                tsp.best_dist = local_best_dist
                
            if local_best_dist < tsp.best_dist:
                tsp.best_dist = local_best_dist
                tsp.best_gen = local_best_gen
                
                tsp.dw.ax.cla()
                tsp.re_draw()
                tsp.dw.plt.pause(0.001)
            else:
                tsp.pop[worst_f_index] = self.best_gen
            print('gen:%d evo,best dist :%s' % (i, self.best_dist))
            
            tsp.pop = tsp.select_pop(tsp.pop)
            tsp.fitness = tsp.get_fitness(tsp.pop)
            
            for j in range(self.pop_size):
                r = np.random.randint(0, self.pop_size - 1)
                if j != r:
                    tsp.pop[j] = tsp.cross(tsp.pop[j], tsp.pop[r])
                    tsp.pop[j] = tsp.mutate(tsp.pop[j])
            self.best_gen = self.EO(self.best_gen)
            tsp.best_dist = tsp.gen_distance(self.best_gen)
    
    #选择种群，优胜劣汰，策略1：低于平均的要替换改变
    def select_pop(self, pop):
        best_f_index = np.argmax(self.fitness)
        av = np.median(self.fitness, axis = 0)
        for i in range(self.pop_size):
            if i != best_f_index and self.fitness[i] < av:
                pi = self.cross(pop[best_f_index], pop[i])
                pi = self.mutate(pi)
                pop[i, :] = pi[:]
        return pop
    
    #选择种群，优胜劣汰，策略2：轮盘赌，适应度低的替换的概率大
    def select_pop2(self, pop):
        probility = self.fitness / self.fitness.sum()
        idx = np.random.choice(np.arange(self.pop_size), size = self.pop_size, replace = True, p = probility)
        n_pop = pop[idx, :]
        return n_pop
    
    #交叉操作
    def cross(self, parent1, parent2):
        #交叉p1,p2的部分基因片段,多点交叉
        if np.random.rand() > self.c_rate:
            return parent1
        index1 = np.random.randint(0, self.city_size - 1)
        index2 = np.random.randint(index1, self.city_size - 1)
        tempGene = parent2[index1 : index2]
        
        newGene = []
        p1len = 0
        for g in parent1:
            if p1len == index1:
                newGene.extend(tempGene)
            if g not in tempGene:
                newGene.append(g)
            p1len += 1
        newGene = np.array(newGene)
        
        #交叉出错后重新生成一个
        if newGene.shape[0] != self.city_size:
            print('cross error')
            return self.create_pop(1)
        return newGene
    
    #基因突变
    def mutate(self, gene):
        if np.random.rand() > self.m_rate:
            return gene
        index1 = np.random.randint(0, self.city_size - 1)
        index2 = np.random.randint(index1, self.city_size - 1)
        newGene = self.reverse_gen(gene, index1, index2)
        
        #突变出错后重新生成一个
        if newGene.shape[0] != self.city_size:
            print('mutation error')
            return self.create_pop(1)
        return newGene
    
    ##函数：翻转基因中i到j之间的基因片段
    def reverse_gen(self, gen, i, j):
        if i >= j:
            return gen
        if j > self.city_size - 1:
            return gen
        parent1 = np.copy(gen)
        tempGene = parent1[i:j]
        newGene = []
        p1len = 0
        for g in parent1:
            if p1len == i:
                newGene.extend(tempGene[::-1])
            if g not in tempGene:
                newGene.append(g)
            p1len += 1
        return np.array(newGene)
    
    #函数：交换基因中i,j值
    def exchange_gen(self, gen, i, j):
        c = gen[j]
        gen[j] = gen[i]
        gen[i] = c
        return gen
    
    def get_fitness(self, pop):
        d = np.array([])
        for i in range(pop.shape[0]):
            gen = pop[i]
            dis = self.gen_distance(gen)
            dis = self.best_dist / dis
            d.append(dis)
        return d
    
    def gen_distance(self, gen):
        distance = 0.0
        for i in range(-1, len(self.citys) - 1):
            index1, index2 = gen[i], gen[i + 1]
            city1, city2 = self.citys[index1], self.citys[index2]
            distance += np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
        return distance

    def ct_distance(self, city1, city2):
        d = np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1] ** 2))
        return d

    
def main():
    tsp = TSP(0.5, 0.1, 100, 500)
    tsp.init()
    tsp.evoluation()
    tsp.re_draw()
    tsp.dw.plt.show()

if __name__ == '__main__':
    main()